In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks import *

In [ ]:
torch.cuda.is_available()

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.1,max_lighting=0.2,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [ ]:
np.random.seed(42)


In [ ]:
# from pathlib import Path
# path = Path("train/")

# print(path)
# for c in data.classes:
#     print(c)
#     verify_images(path/c, delete=True)

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

In [ ]:
loss_func = FocalLoss(gamma=2.)

In [ ]:
src = (ImageList.from_folder(path='train').split_by_rand_pct(0.2).label_from_folder())

In [ ]:
src

In [ ]:
data = src.transform(tfms,size=256).databunch(bs=12).normalize(imagenet_stats)

In [ ]:
data.c

In [ ]:
data

In [ ]:
learn = cnn_learner(data, models.densenet161, metrics=[error_rate, accuracy, FBeta(beta=1,average='macro')],loss_func=loss_func)


In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=7e-3

In [ ]:
learn.fit_one_cycle(2,lr)

In [ ]:
lr=1e-3

In [ ]:
learn.fit_one_cycle(2,lr)

In [ ]:
learn.fit_one_cycle(2,slice(1e-6,lr/10))

In [ ]:
data = src.transform(tfms,size=512).databunch(bs=6).normalize(imagenet_stats)

In [ ]:
learn.data = data

In [ ]:
learn.lr_find()

In [ ]:
learn

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(,slice(1e-6,lr/10))

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60,normalize=True)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60,normalize=False)

In [ ]:
interp.most_confused(min_val=2)
